In [64]:
# just importing a random selection of things; we can fix this later
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LinearRegression, LogisticRegressionCV
%matplotlib inline

from sklearn.tree import DecisionTreeRegressor

from sklearn.model_selection import cross_val_score

from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.grid_search import GridSearchCV

from pprint import pprint 

from sklearn.ensemble import BaggingClassifier
from sklearn import model_selection
from random import seed
from sklearn.model_selection import cross_val_predict

from sklearn.ensemble import RandomForestClassifier
from itertools import product
from collections import OrderedDict
from itertools import product

from sklearn import model_selection
from sklearn.svm import SVC
from sklearn import neighbors
from sklearn.ensemble import VotingClassifier

from sklearn.model_selection import train_test_split
from __future__ import print_function # For Python 2 / 3 compatability
from sklearn.metrics import confusion_matrix
import itertools

import json
from copy import copy

import warnings
from sklearn import preprocessing
%autosave 300
warnings.filterwarnings("ignore")

Autosaving every 300 seconds


# Data Cleaning and Preliminary Exploration

## Preliminary Exploration

In [65]:
# load ADNIMERGE.csv, the main dataset (we will add our own variables to it). 
ADNI_merge = pd.read_csv('ADNIMERGE.csv')
ADNI_merge.head()

# select only the rows that correspond to ADN2
ADNI2= ADNI_merge[ADNI_merge['COLPROT']=='ADNI2']
print(ADNI2.shape)


(6937, 94)


In [66]:
# #### NOTE: this is new code (11.23.17) because missing values are also denoted by "-4" and "-1". 
# #### Please see the training powerpoint for more info: https://adni.loni.usc.edu/wp-content/uploads/2012/08/slide_data_training_part2_reduced-size.pdf

# # checking out the number of records for patients in ADNI2 phase, compared with other phases
# ADMCPATIENTDRUGCLASSES_20170512 = pd.read_csv('ADMCPATIENTDRUGCLASSES_20170512.csv')
# print(len(ADMCPATIENTDRUGCLASSES_20170512['Phase']))
# print(len(ADMCPATIENTDRUGCLASSES_20170512[ADMCPATIENTDRUGCLASSES_20170512['Phase']=='ADNI2']))
# print(len(ADMCPATIENTDRUGCLASSES_20170512[ADMCPATIENTDRUGCLASSES_20170512['Phase']=='ADNI3']))
# print(len(ADMCPATIENTDRUGCLASSES_20170512[ADMCPATIENTDRUGCLASSES_20170512['Phase']=='ADNIGO']))

# ADMCPATIENTDRUGCLASSES_20170512.head()

# drug_adni2 = ADMCPATIENTDRUGCLASSES_20170512[ADMCPATIENTDRUGCLASSES_20170512['Phase']=='ADNI2']
# drug_adni2.shape

# drug_adni2.columns

# drug_adni2 = drug_adni2.rename(columns={'VISCODE2': 'VISCODE'})
# drug_adni2.columns
# pd.merge(ADNI2,drug_adni2, on=['RID', 'VISCODE'])

In [67]:
# It seems there were some 'unknown'..I replaced them with nan
import numpy as np
ADNI2 = ADNI2.replace('Unknown', np.nan)
ADNI2 = ADNI2.replace(-1, np.nan)
ADNI2 = ADNI2.replace(-4, np.nan)

# From now on DX


In [142]:
ADNI2_dx = ADNI2.copy()

In [143]:
ADNI2_dx.columns

Index(['RID', 'PTID', 'VISCODE', 'SITE', 'COLPROT', 'ORIGPROT', 'EXAMDATE',
       'DX_bl', 'AGE', 'PTGENDER', 'PTEDUCAT', 'PTETHCAT', 'PTRACCAT',
       'PTMARRY', 'APOE4', 'FDG', 'PIB', 'AV45', 'CDRSB', 'ADAS11', 'ADAS13',
       'MMSE', 'RAVLT_immediate', 'RAVLT_learning', 'RAVLT_forgetting',
       'RAVLT_perc_forgetting', 'FAQ', 'MOCA', 'EcogPtMem', 'EcogPtLang',
       'EcogPtVisspat', 'EcogPtPlan', 'EcogPtOrgan', 'EcogPtDivatt',
       'EcogPtTotal', 'EcogSPMem', 'EcogSPLang', 'EcogSPVisspat', 'EcogSPPlan',
       'EcogSPOrgan', 'EcogSPDivatt', 'EcogSPTotal', 'FLDSTRENG', 'FSVERSION',
       'Ventricles', 'Hippocampus', 'WholeBrain', 'Entorhinal', 'Fusiform',
       'MidTemp', 'ICV', 'DX', 'EXAMDATE_bl', 'CDRSB_bl', 'ADAS11_bl',
       'ADAS13_bl', 'MMSE_bl', 'RAVLT_immediate_bl', 'RAVLT_learning_bl',
       'RAVLT_forgetting_bl', 'RAVLT_perc_forgetting_bl', 'FAQ_bl',
       'FLDSTRENG_bl', 'FSVERSION_bl', 'Ventricles_bl', 'Hippocampus_bl',
       'WholeBrain_bl', 'Entorhinal_bl

In [144]:
ADNI2_dx.shape

(6937, 94)

In [145]:
#Drop the ones w/o bl
#I kept 'RID', 'PTID', 'VISCODE', 'COLPROT','DX_bl'
#'AGE', 'PTGENDER', 'PTEDUCAT', 'PTETHCAT', 'PTRACCAT',
#'PTMARRY', 'APOE4', 'FDG', , ADAS11_bl',
# 'ADAS13_bl', 'CDRSB_bl', 'MMSE_bl',
#'RAVLT_immediate_bl', 'RAVLT_learning_bl',
#'RAVLT_forgetting_bl', 'RAVLT_perc_forgetting_bl', 'FAQ_bl',
#'FLDSTRENG_bl', , 'Ventricles_bl', 'Hippocampus_bl',
#'WholeBrain_bl', 'Entorhinal_bl', 'Fusiform_bl', 'MidTemp_bl', 'ICV_bl',
#'MOCA_bl', 'EcogPtMem_bl', 'EcogPtLang_bl', 'EcogPtVisspat_bl',
#'EcogPtPlan_bl', 'EcogPtOrgan_bl', 'EcogPtDivatt_bl', 'EcogPtTotal_bl',
#'EcogSPMem_bl', 'EcogSPLang_bl', 'EcogSPVisspat_bl', 'EcogSPPlan_bl',
#'EcogSPOrgan_bl', 'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'FDG_bl',
#'PIB_bl', 'AV45_bl'


ADNI2_dx= ADNI2_dx.drop(labels = ['Month_bl', 'Month', 'M'],axis=1)

In [146]:
#Remove rows where DX is nan
print(ADNI2_dx.shape)
#ADNI2_dx_clean = ADNI2_dx[ADNI2_dx['DX'] != np.nan]
ADNI2_dx= ADNI2_dx[pd.notnull(ADNI2_dx['DX'])]
print(ADNI2_dx.shape)

(6937, 91)
(4568, 91)


In [147]:
# calculate the % of each column that is NaNs and print in descending order
print((ADNI2_dx.isnull().sum()/len(ADNI2_dx['DX'])).sort_values(ascending=False))

PIB                         1.000000
PIB_bl                      0.994527
FDG                         0.707531
AV45                        0.591287
FLDSTRENG                   0.437391
MidTemp                     0.430604
Fusiform                    0.430604
Entorhinal                  0.430604
Ventricles                  0.374343
Hippocampus                 0.362741
WholeBrain                  0.338222
FSVERSION                   0.298161
ICV                         0.298161
EcogSPOrgan_bl              0.232706
EcogSPDivatt_bl             0.212785
EcogPtOrgan_bl              0.210158
EcogSPVisspat_bl            0.207312
EcogSPPlan_bl               0.202277
MOCA_bl                     0.200525
AV45_bl                     0.198993
EcogPtDivatt_bl             0.198117
FLDSTRENG_bl                0.197898
EcogPtVisspat_bl            0.196804
EcogSPTotal_bl              0.196147
EcogSPMem_bl                0.195709
EcogSPLang_bl               0.194834
EcogPtPlan_bl               0.193520
E

In [148]:
#Drop 40% or more missing
ADNI2_dx= ADNI2_dx.drop(['PIB_bl','PIB', 'AV45','FLDSTRENG','MidTemp','Entorhinal', 'Fusiform','FDG'], axis=1)
print((ADNI2_dx.isnull().sum()/len(ADNI2_dx['DX'])).sort_values(ascending=False))

Ventricles                  0.374343
Hippocampus                 0.362741
WholeBrain                  0.338222
FSVERSION                   0.298161
ICV                         0.298161
EcogSPOrgan_bl              0.232706
EcogSPDivatt_bl             0.212785
EcogPtOrgan_bl              0.210158
EcogSPVisspat_bl            0.207312
EcogSPPlan_bl               0.202277
MOCA_bl                     0.200525
AV45_bl                     0.198993
EcogPtDivatt_bl             0.198117
FLDSTRENG_bl                0.197898
EcogPtVisspat_bl            0.196804
EcogSPTotal_bl              0.196147
EcogSPMem_bl                0.195709
EcogSPLang_bl               0.194834
EcogPtPlan_bl               0.193520
EcogPtLang_bl               0.193301
EcogPtTotal_bl              0.193301
EcogPtMem_bl                0.193082
Fusiform_bl                 0.123687
Entorhinal_bl               0.123687
MidTemp_bl                  0.123687
Hippocampus_bl              0.111208
FDG_bl                      0.101795
E

In [149]:
encode = preprocessing.LabelEncoder()
ADNI2_dx.loc[:, 'DX'] = encode.fit_transform(ADNI2_dx['DX'])

In [150]:
print(ADNI2_dx.shape)
ADNI2_dx['FSVERSION'].value_counts()

(4568, 83)


Cross-Sectional FreeSurfer (5.1)                       2967
Cross-Sectional FreeSurfer (FreeSurfer Version 4.3)     239
Name: FSVERSION, dtype: int64

In [151]:
print(ADNI2_dx.shape)
ADNI2_dx['FLDSTRENG_bl'].value_counts()

(4568, 83)


3 Tesla MRI      2793
1.5 Tesla MRI     871
Name: FLDSTRENG_bl, dtype: int64

In [152]:
print(ADNI2_dx.shape)
ADNI2_dx['FSVERSION_bl'].value_counts()

(4568, 83)


Cross-Sectional FreeSurfer (5.1)                       3640
Cross-Sectional FreeSurfer (FreeSurfer Version 4.3)     875
Name: FSVERSION_bl, dtype: int64

In [153]:
ADNI2_dx.columns

Index(['RID', 'PTID', 'VISCODE', 'SITE', 'COLPROT', 'ORIGPROT', 'EXAMDATE',
       'DX_bl', 'AGE', 'PTGENDER', 'PTEDUCAT', 'PTETHCAT', 'PTRACCAT',
       'PTMARRY', 'APOE4', 'CDRSB', 'ADAS11', 'ADAS13', 'MMSE',
       'RAVLT_immediate', 'RAVLT_learning', 'RAVLT_forgetting',
       'RAVLT_perc_forgetting', 'FAQ', 'MOCA', 'EcogPtMem', 'EcogPtLang',
       'EcogPtVisspat', 'EcogPtPlan', 'EcogPtOrgan', 'EcogPtDivatt',
       'EcogPtTotal', 'EcogSPMem', 'EcogSPLang', 'EcogSPVisspat', 'EcogSPPlan',
       'EcogSPOrgan', 'EcogSPDivatt', 'EcogSPTotal', 'FSVERSION', 'Ventricles',
       'Hippocampus', 'WholeBrain', 'ICV', 'DX', 'EXAMDATE_bl', 'CDRSB_bl',
       'ADAS11_bl', 'ADAS13_bl', 'MMSE_bl', 'RAVLT_immediate_bl',
       'RAVLT_learning_bl', 'RAVLT_forgetting_bl', 'RAVLT_perc_forgetting_bl',
       'FAQ_bl', 'FLDSTRENG_bl', 'FSVERSION_bl', 'Ventricles_bl',
       'Hippocampus_bl', 'WholeBrain_bl', 'Entorhinal_bl', 'Fusiform_bl',
       'MidTemp_bl', 'ICV_bl', 'MOCA_bl', 'EcogPtMem_bl', 'Ec

In [154]:
#hot-one encoding for categoricla and clean

categorical_predictors = ['PTETHCAT', 'PTRACCAT', 'PTMARRY','APOE4', 'PTGENDER', 'DX_bl']
categorical_predictors_drop = ['FSVERSION', 'FSVERSION_bl','FLDSTRENG_bl']



''' Remove all categorical data with nan (only ~0.5%) '''
missing_rows = np.nonzero(np.sum(ADNI2_dx[categorical_predictors].isnull(), axis=1))
ADNI2_dx = ADNI2_dx.drop(ADNI2_dx.index[missing_rows[0]])

'''' Hot-One Encoding of Categorical Predictors '''
ADNI2_dx_categorical_nodrop = pd.get_dummies(ADNI2_dx[categorical_predictors] , columns=categorical_predictors, prefix = categorical_predictors , drop_first=False)
ADNI2_dx_categorical_drop = pd.get_dummies(ADNI2_dx[categorical_predictors_drop] , columns=categorical_predictors_drop, prefix = categorical_predictors_drop , drop_first=False)
ADNI2_dx= pd.get_dummies(ADNI2_dx, columns=categorical_predictors, prefix = categorical_predictors , drop_first=False)
ADNI2_dx= pd.get_dummies(ADNI2_dx, columns=categorical_predictors_drop, prefix = categorical_predictors_drop , drop_first=True)
print(ADNI2_dx.shape)


(4511, 99)


In [155]:
print(ADNI2_dx_categorical_nodrop.shape)
print(ADNI2_dx_categorical_drop.shape)

(4511, 22)
(4511, 6)


In [156]:
ADNI2_dx_categorical = pd.concat([ADNI2_dx_categorical_nodrop, ADNI2_dx_categorical_drop],axis=1)
print(ADNI2_dx_categorical.shape)

(4511, 28)


In [157]:
print(ADNI2_dx.columns)

Index(['RID', 'PTID', 'VISCODE', 'SITE', 'COLPROT', 'ORIGPROT', 'EXAMDATE',
       'AGE', 'PTEDUCAT', 'CDRSB', 'ADAS11', 'ADAS13', 'MMSE',
       'RAVLT_immediate', 'RAVLT_learning', 'RAVLT_forgetting',
       'RAVLT_perc_forgetting', 'FAQ', 'MOCA', 'EcogPtMem', 'EcogPtLang',
       'EcogPtVisspat', 'EcogPtPlan', 'EcogPtOrgan', 'EcogPtDivatt',
       'EcogPtTotal', 'EcogSPMem', 'EcogSPLang', 'EcogSPVisspat', 'EcogSPPlan',
       'EcogSPOrgan', 'EcogSPDivatt', 'EcogSPTotal', 'Ventricles',
       'Hippocampus', 'WholeBrain', 'ICV', 'DX', 'EXAMDATE_bl', 'CDRSB_bl',
       'ADAS11_bl', 'ADAS13_bl', 'MMSE_bl', 'RAVLT_immediate_bl',
       'RAVLT_learning_bl', 'RAVLT_forgetting_bl', 'RAVLT_perc_forgetting_bl',
       'FAQ_bl', 'Ventricles_bl', 'Hippocampus_bl', 'WholeBrain_bl',
       'Entorhinal_bl', 'Fusiform_bl', 'MidTemp_bl', 'ICV_bl', 'MOCA_bl',
       'EcogPtMem_bl', 'EcogPtLang_bl', 'EcogPtVisspat_bl', 'EcogPtPlan_bl',
       'EcogPtOrgan_bl', 'EcogPtDivatt_bl', 'EcogPtTotal_bl', 'Eco

### Removing All Missing Data

In [158]:
ADNI2_dx_remove = ADNI2_dx.copy()
print('Shape before delteing missing', ADNI2_dx_remove.shape)
ADNI2_dx_remove.dropna(axis=0, inplace=True)
print('Shape before delteing missing', ADNI2_dx_remove.shape) 
print('Data contains NaN?', ADNI2_dx_remove.isnull().any().any())

Shape before delteing missing (4511, 99)
Shape before delteing missing (1536, 99)
Data contains NaN? False


In [159]:
ADNI2_dx_remove.isnull().sum()

RID                                                                 0
PTID                                                                0
VISCODE                                                             0
SITE                                                                0
COLPROT                                                             0
ORIGPROT                                                            0
EXAMDATE                                                            0
AGE                                                                 0
PTEDUCAT                                                            0
CDRSB                                                               0
ADAS11                                                              0
ADAS13                                                              0
MMSE                                                                0
RAVLT_immediate                                                     0
RAVLT_learning      

In [160]:
ADNI2_dx_remove.head()

,RID,PTID,VISCODE,SITE,COLPROT,ORIGPROT,EXAMDATE,AGE,PTEDUCAT,CDRSB,...,PTGENDER_Female,PTGENDER_Male,DX_bl_AD,DX_bl_CN,DX_bl_EMCI,DX_bl_LMCI,DX_bl_SMC,FSVERSION_Cross-Sectional FreeSurfer (FreeSurfer Version 4.3),FSVERSION_bl_Cross-Sectional FreeSurfer (FreeSurfer Version 4.3),FLDSTRENG_bl_3 Tesla MRI
91,5280,100_S_5280,bl,100,ADNI2,ADNI2,2013-09-17,67.5,16,0.0,...,0,1,0,0,0,0,1,0,0,1
92,5279,082_S_5279,bl,82,ADNI2,ADNI2,2013-10-23,68.5,20,0.0,...,0,1,0,0,0,0,1,0,0,1
142,5278,082_S_5278,bl,82,ADNI2,ADNI2,2013-09-04,80.2,19,0.0,...,0,1,0,0,0,0,1,0,0,1
153,5277,027_S_5277,m24,27,ADNI2,ADNI2,2015-08-31,71.8,15,0.5,...,1,0,0,0,0,0,1,0,0,1
174,5277,027_S_5277,bl,27,ADNI2,ADNI2,2013-08-28,71.8,15,0.5,...,1,0,0,0,0,0,1,0,0,1


In [161]:
ADNI2_dx_remove.columns

Index(['RID', 'PTID', 'VISCODE', 'SITE', 'COLPROT', 'ORIGPROT', 'EXAMDATE',
       'AGE', 'PTEDUCAT', 'CDRSB', 'ADAS11', 'ADAS13', 'MMSE',
       'RAVLT_immediate', 'RAVLT_learning', 'RAVLT_forgetting',
       'RAVLT_perc_forgetting', 'FAQ', 'MOCA', 'EcogPtMem', 'EcogPtLang',
       'EcogPtVisspat', 'EcogPtPlan', 'EcogPtOrgan', 'EcogPtDivatt',
       'EcogPtTotal', 'EcogSPMem', 'EcogSPLang', 'EcogSPVisspat', 'EcogSPPlan',
       'EcogSPOrgan', 'EcogSPDivatt', 'EcogSPTotal', 'Ventricles',
       'Hippocampus', 'WholeBrain', 'ICV', 'DX', 'EXAMDATE_bl', 'CDRSB_bl',
       'ADAS11_bl', 'ADAS13_bl', 'MMSE_bl', 'RAVLT_immediate_bl',
       'RAVLT_learning_bl', 'RAVLT_forgetting_bl', 'RAVLT_perc_forgetting_bl',
       'FAQ_bl', 'Ventricles_bl', 'Hippocampus_bl', 'WholeBrain_bl',
       'Entorhinal_bl', 'Fusiform_bl', 'MidTemp_bl', 'ICV_bl', 'MOCA_bl',
       'EcogPtMem_bl', 'EcogPtLang_bl', 'EcogPtVisspat_bl', 'EcogPtPlan_bl',
       'EcogPtOrgan_bl', 'EcogPtDivatt_bl', 'EcogPtTotal_bl', 'Eco

In [162]:
ADNI2_dx_remove.to_csv(path_or_buf='ADNI2_dx_remove_all_missing.csv', index=False)

## Model-Based Imputation 
- Drop around 5% of Categorical predictors and use Linear Regression for missing continuous predictors

In [163]:
def split (data, percentage):
    np.random.seed(9001) 
    msk = np.random.rand(len(data)) < percentage
    train = data[msk]
    test = data[~msk]
    return train, test

In [164]:
ADNI2_dx_impute = ADNI2_dx.copy()

In [165]:
ADNI2_dx_impute.columns

Index(['RID', 'PTID', 'VISCODE', 'SITE', 'COLPROT', 'ORIGPROT', 'EXAMDATE',
       'AGE', 'PTEDUCAT', 'CDRSB', 'ADAS11', 'ADAS13', 'MMSE',
       'RAVLT_immediate', 'RAVLT_learning', 'RAVLT_forgetting',
       'RAVLT_perc_forgetting', 'FAQ', 'MOCA', 'EcogPtMem', 'EcogPtLang',
       'EcogPtVisspat', 'EcogPtPlan', 'EcogPtOrgan', 'EcogPtDivatt',
       'EcogPtTotal', 'EcogSPMem', 'EcogSPLang', 'EcogSPVisspat', 'EcogSPPlan',
       'EcogSPOrgan', 'EcogSPDivatt', 'EcogSPTotal', 'Ventricles',
       'Hippocampus', 'WholeBrain', 'ICV', 'DX', 'EXAMDATE_bl', 'CDRSB_bl',
       'ADAS11_bl', 'ADAS13_bl', 'MMSE_bl', 'RAVLT_immediate_bl',
       'RAVLT_learning_bl', 'RAVLT_forgetting_bl', 'RAVLT_perc_forgetting_bl',
       'FAQ_bl', 'Ventricles_bl', 'Hippocampus_bl', 'WholeBrain_bl',
       'Entorhinal_bl', 'Fusiform_bl', 'MidTemp_bl', 'ICV_bl', 'MOCA_bl',
       'EcogPtMem_bl', 'EcogPtLang_bl', 'EcogPtVisspat_bl', 'EcogPtPlan_bl',
       'EcogPtOrgan_bl', 'EcogPtDivatt_bl', 'EcogPtTotal_bl', 'Eco

In [166]:
all_predictors = ['RID', 'PTID', 'VISCODE', 'SITE', 'COLPROT', 'ORIGPROT', 'EXAMDATE',
       'DX_bl', 'AGE', 'PTEDUCAT', 'CDRSB', 'ADAS11', 'ADAS13', 'MMSE',
       'RAVLT_immediate', 'RAVLT_learning', 'RAVLT_forgetting',
       'RAVLT_perc_forgetting', 'FAQ', 'MOCA', 'EcogPtMem', 'EcogPtLang',
       'EcogPtVisspat', 'EcogPtPlan', 'EcogPtOrgan', 'EcogPtDivatt',
       'EcogPtTotal', 'EcogSPMem', 'EcogSPLang', 'EcogSPVisspat', 'EcogSPPlan',
       'EcogSPOrgan', 'EcogSPDivatt', 'EcogSPTotal', 'Ventricles',
       'Hippocampus', 'WholeBrain', 'ICV', 'DX', 'EXAMDATE_bl', 'CDRSB_bl',
       'ADAS11_bl', 'ADAS13_bl', 'MMSE_bl', 'RAVLT_immediate_bl',
       'RAVLT_learning_bl', 'RAVLT_forgetting_bl', 'RAVLT_perc_forgetting_bl',
       'FAQ_bl', 'Ventricles_bl', 'Hippocampus_bl', 'WholeBrain_bl',
       'Entorhinal_bl', 'Fusiform_bl', 'MidTemp_bl', 'ICV_bl', 'MOCA_bl',
       'EcogPtMem_bl', 'EcogPtLang_bl', 'EcogPtVisspat_bl', 'EcogPtPlan_bl',
       'EcogPtOrgan_bl', 'EcogPtDivatt_bl', 'EcogPtTotal_bl', 'EcogSPMem_bl',
       'EcogSPLang_bl', 'EcogSPVisspat_bl', 'EcogSPPlan_bl', 'EcogSPOrgan_bl',
       'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'FDG_bl', 'AV45_bl', 'Years_bl',
       'update_stamp', 'PTETHCAT_Hisp/Latino', 'PTETHCAT_Not Hisp/Latino',
       'PTRACCAT_Am Indian/Alaskan', 'PTRACCAT_Asian', 'PTRACCAT_Black',
       'PTRACCAT_Hawaiian/Other PI', 'PTRACCAT_More than one',
       'PTRACCAT_White', 'PTMARRY_Divorced', 'PTMARRY_Married',
       'PTMARRY_Never married', 'PTMARRY_Widowed', 'APOE4_0.0', 'APOE4_1.0',
       'APOE4_2.0', 'PTGENDER_Female', 'PTGENDER_Male',
       'FSVERSION_Cross-Sectional FreeSurfer (FreeSurfer Version 4.3)',
       'FSVERSION_bl_Cross-Sectional FreeSurfer (FreeSurfer Version 4.3)',
       'FLDSTRENG_bl_3 Tesla MRI']


         
continuous_predictors = ['AGE', 
       'PTEDUCAT', 'CDRSB_bl', 'ADAS11_bl', 'ADAS13_bl', 'MMSE_bl',
       'RAVLT_immediate_bl', 'RAVLT_learning_bl', 'RAVLT_forgetting_bl',
       'RAVLT_perc_forgetting_bl', 'FAQ_bl',
       'Ventricles_bl', 'Hippocampus_bl', 'WholeBrain_bl', 'Entorhinal_bl',
       'Fusiform_bl', 'MidTemp_bl', 'ICV_bl', 'MOCA_bl', 'EcogPtMem_bl',
       'EcogPtLang_bl', 'EcogPtVisspat_bl', 'EcogPtPlan_bl', 'EcogPtOrgan_bl',
       'EcogPtDivatt_bl', 'EcogPtTotal_bl', 'EcogSPMem_bl', 'EcogSPLang_bl',
       'EcogSPVisspat_bl', 'EcogSPPlan_bl', 'EcogSPOrgan_bl',
       'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'FDG_bl', 'AV45_bl',
        'CDRSB', 'ADAS11', 'ADAS13','MMSE','FAQ', 'MOCA', 'EcogPtMem', 'EcogPtLang',
       'EcogPtVisspat', 'EcogPtPlan', 'EcogPtOrgan', 'EcogPtDivatt',
       'EcogPtTotal', 'EcogSPMem', 'EcogSPLang', 'EcogSPVisspat', 'EcogSPPlan',
       'EcogSPOrgan', 'EcogSPDivatt', 'EcogSPTotal', 'Ventricles',
       'Hippocampus', 'WholeBrain', 'ICV','Years_bl',
        'RAVLT_immediate', 'RAVLT_learning', 'RAVLT_forgetting',
        'RAVLT_perc_forgetting',
       ]

In [167]:
ADNI2_dx.shape

(4511, 99)

In [168]:
''' Create ADNI2_missing for imputation '''
ADNI2_dx_impute_remove_categorical = ADNI2_dx_impute[continuous_predictors]

''' Create the Dataframe for imputation '''
ADNI2_dx_missing = pd.concat([ADNI2_dx_impute_remove_categorical, ADNI2_dx_categorical], axis=1)


In [169]:
print(ADNI2_dx_impute_remove_categorical.shape)
print(ADNI2_dx_categorical.shape)
print(ADNI2_dx.shape)

# I did not consider 
#'RID', 'PTID', 'VISCODE', 'SITE', 'COLPROT', 'ORIGPROT', 'EXAMDATE',  EXAMDATE_bl',

(4511, 64)
(4511, 28)
(4511, 99)


In [171]:
ADNI2_dx_missing['DX']=ADNI2_dx_impute['DX']
ADNI2_dx_missing.head()

,AGE,PTEDUCAT,CDRSB_bl,ADAS11_bl,ADAS13_bl,MMSE_bl,RAVLT_immediate_bl,RAVLT_learning_bl,RAVLT_forgetting_bl,RAVLT_perc_forgetting_bl,...,DX_bl_EMCI,DX_bl_LMCI,DX_bl_SMC,FSVERSION_Cross-Sectional FreeSurfer (5.1),FSVERSION_Cross-Sectional FreeSurfer (FreeSurfer Version 4.3),FSVERSION_bl_Cross-Sectional FreeSurfer (5.1),FSVERSION_bl_Cross-Sectional FreeSurfer (FreeSurfer Version 4.3),FLDSTRENG_bl_1.5 Tesla MRI,FLDSTRENG_bl_3 Tesla MRI,DX
58,66.9,17,0.0,10.0,15.0,29,42.0,5.0,2.0,20.0,...,0,0,1,1,0,1,0,0,1,0
59,67.5,16,0.0,3.0,5.0,30,42.0,7.0,2.0,20.0,...,0,0,1,0,0,1,0,0,1,0
68,67.5,16,0.0,3.0,5.0,30,42.0,7.0,2.0,20.0,...,0,0,1,0,0,1,0,0,1,0
91,67.5,16,0.0,3.0,5.0,30,42.0,7.0,2.0,20.0,...,0,0,1,1,0,1,0,0,1,0
92,68.5,20,0.0,4.0,7.0,30,61.0,6.0,3.0,20.0,...,0,0,1,1,0,1,0,0,1,0


In [172]:
print(ADNI2_dx_impute_remove_categorical.shape)

(4511, 64)


In [173]:
print(ADNI2_dx_missing.columns)

Index(['AGE', 'PTEDUCAT', 'CDRSB_bl', 'ADAS11_bl', 'ADAS13_bl', 'MMSE_bl',
       'RAVLT_immediate_bl', 'RAVLT_learning_bl', 'RAVLT_forgetting_bl',
       'RAVLT_perc_forgetting_bl', 'FAQ_bl', 'Ventricles_bl', 'Hippocampus_bl',
       'WholeBrain_bl', 'Entorhinal_bl', 'Fusiform_bl', 'MidTemp_bl', 'ICV_bl',
       'MOCA_bl', 'EcogPtMem_bl', 'EcogPtLang_bl', 'EcogPtVisspat_bl',
       'EcogPtPlan_bl', 'EcogPtOrgan_bl', 'EcogPtDivatt_bl', 'EcogPtTotal_bl',
       'EcogSPMem_bl', 'EcogSPLang_bl', 'EcogSPVisspat_bl', 'EcogSPPlan_bl',
       'EcogSPOrgan_bl', 'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'FDG_bl',
       'AV45_bl', 'CDRSB', 'ADAS11', 'ADAS13', 'MMSE', 'FAQ', 'MOCA',
       'EcogPtMem', 'EcogPtLang', 'EcogPtVisspat', 'EcogPtPlan', 'EcogPtOrgan',
       'EcogPtDivatt', 'EcogPtTotal', 'EcogSPMem', 'EcogSPLang',
       'EcogSPVisspat', 'EcogSPPlan', 'EcogSPOrgan', 'EcogSPDivatt',
       'EcogSPTotal', 'Ventricles', 'Hippocampus', 'WholeBrain', 'ICV',
       'Years_bl', 'RAVLT_immediate',

In [188]:
full_dataframe =  copy(ADNI2_dx_missing)
full_dataframe_train, full_dataframe_test = split(full_dataframe,0.75)
ADNI2_dx_missing_train, ADNI2_dx_missing_test = split(ADNI2_dx_missing,0.75)

In [189]:
#For train
missing_rows = []
for i in range(ADNI2_dx_missing_train.shape[0]):
    if pd.isnull(ADNI2_dx_missing_train.values[i]).any():
        missing_rows.append(i)
full_dataframe_train = full_dataframe_train.drop(full_dataframe_train.index[missing_rows], axis=0);

print('The origin number of data: {}'.format(len(ADNI2_dx_missing_train)))
print('The number of data that contains missing entries: {}'.format(len(missing_rows)))
print('The dimension of full dataframe: {}'.format(full_dataframe_train.shape))

''' To check that full_dataframe has no missing values '''
print(np.sum(np.sum(full_dataframe_train.isnull())))

The origin number of data: 3397
The number of data that contains missing entries: 2266
The dimension of full dataframe: (1131, 93)
0


In [190]:
''' Model-Based Imputation - Linear Regression for train'''

lr = LinearRegression(fit_intercept = True)


for i in range(ADNI2_dx_missing_train.shape[0]):
    if ADNI2_dx_missing_train.iloc[i, :].isnull().any():
        missing_columns = ADNI2_dx_missing_train.columns[ADNI2_dx_missing_train.iloc[i, :].isnull()]
        #print(i, missing_columns)
        y_imp_train = full_dataframe_train[missing_columns]
        X_imp_train = full_dataframe_train.drop(missing_columns, 1)
#         print(y_imp.shape)
#         print(X_imp.shape)
        lr.fit(X_imp_train,y_imp_train)
        X_missing_train = ADNI2_dx_missing_train.iloc[i, :].drop(missing_columns)
        y_hat_train = lr.predict(X_missing_train)
        ADNI2_dx_missing_train.loc[ADNI2_dx_missing_train.index[i], missing_columns] = y_hat_train[0] 
     

In [191]:
ADNI2_dx_missing_train.isnull().sum()

AGE                                                                 0
PTEDUCAT                                                            0
CDRSB_bl                                                            0
ADAS11_bl                                                           0
ADAS13_bl                                                           0
MMSE_bl                                                             0
RAVLT_immediate_bl                                                  0
RAVLT_learning_bl                                                   0
RAVLT_forgetting_bl                                                 0
RAVLT_perc_forgetting_bl                                            0
FAQ_bl                                                              0
Ventricles_bl                                                       0
Hippocampus_bl                                                      0
WholeBrain_bl                                                       0
Entorhinal_bl       

In [192]:
#For test
missing_rows_test = []
for i in range(ADNI2_dx_missing_test.shape[0]):
    if pd.isnull(ADNI2_dx_missing_test.values[i]).any():
        missing_rows_test.append(i)
full_dataframe_test = full_dataframe_test.drop(full_dataframe_test.index[missing_rows_test], axis=0);

print('The origin number of data: {}'.format(len(ADNI2_dx_missing_test)))
print('The number of data that contains missing entries: {}'.format(len(missing_rows_test)))
print('The dimension of full dataframe: {}'.format(full_dataframe_test.shape))

''' To check that full_dataframe has no missing values '''
print(np.sum(np.sum(full_dataframe_test.isnull())))

The origin number of data: 1114
The number of data that contains missing entries: 709
The dimension of full dataframe: (405, 93)
0


In [194]:
#Model-Based Imputation - Linear Regression for test

lr = LinearRegression(fit_intercept = True)

for i in range(ADNI2_dx_missing_test.shape[0]):
    if ADNI2_dx_missing_test.iloc[i, :].isnull().any():
        missing_columns = ADNI2_dx_missing_test.columns[ADNI2_dx_missing_test.iloc[i, :].isnull()]
        y_imp_test = full_dataframe_test[missing_columns]
        X_imp_test = full_dataframe_test.drop(missing_columns, 1)
        #print(i, missing_columns)
#         print(y_imp.shape)
#         print(X_imp.shape)
        lr.fit(X_imp_test,y_imp_test)
        X_missing_test = ADNI2_dx_missing_test.iloc[i, :].drop(missing_columns)
        y_hat_test = lr.predict(X_missing_test)
        ADNI2_dx_missing_test.loc[ADNI2_dx_missing_test.index[i], missing_columns] = y_hat_test[0] 
     

In [198]:
ADNI2_dx_missing_train.isnull().sum()

AGE                                                                 0
PTEDUCAT                                                            0
CDRSB_bl                                                            0
ADAS11_bl                                                           0
ADAS13_bl                                                           0
MMSE_bl                                                             0
RAVLT_immediate_bl                                                  0
RAVLT_learning_bl                                                   0
RAVLT_forgetting_bl                                                 0
RAVLT_perc_forgetting_bl                                            0
FAQ_bl                                                              0
Ventricles_bl                                                       0
Hippocampus_bl                                                      0
WholeBrain_bl                                                       0
Entorhinal_bl       

In [197]:
print(ADNI2_dx_missing_train.shape)
print(ADNI2_dx_missing_test.shape)
print(ADNI2_dx_remove.shape)

(3397, 93)
(1114, 93)
(1536, 99)


In [41]:
ADNI2_bl_missing_test.isnull().sum()

AGE                           0
PTEDUCAT                      0
CDRSB_bl                      0
ADAS11_bl                     0
ADAS13_bl                     0
MMSE_bl                       0
RAVLT_immediate_bl            0
RAVLT_learning_bl             0
RAVLT_forgetting_bl           0
RAVLT_perc_forgetting_bl      0
FAQ_bl                        0
Ventricles_bl                 0
Hippocampus_bl                0
WholeBrain_bl                 0
Entorhinal_bl                 0
Fusiform_bl                   0
MidTemp_bl                    0
ICV_bl                        0
MOCA_bl                       0
EcogPtMem_bl                  0
EcogPtLang_bl                 0
EcogPtVisspat_bl              0
EcogPtPlan_bl                 0
EcogPtOrgan_bl                0
EcogPtDivatt_bl               0
EcogPtTotal_bl                0
EcogSPMem_bl                  0
EcogSPLang_bl                 0
EcogSPVisspat_bl              0
EcogSPPlan_bl                 0
EcogSPOrgan_bl                0
EcogSPDi

In [200]:
ADNI2_dx_missing_train.to_csv(path_or_buf='ADNI2_dx_impute_train.csv', index=False)
ADNI2_dx_missing_test.to_csv(path_or_buf='ADNI2_dx_impute_test.csv', index=False)

# Below are More Preliminary Exploration by Carlo

                                                 **STRATEGY**

'RID',                    They should not be used for prediction, I'd keep them just for merging the prediction?
'PTID',                   They should not be used for prediction, I'd keep them just for merging the prediction?      
'VISCODE'                 They should not be used for prediction, I'd keep them just for merging the prediction?        'SITE'                    Shall we discard for the prediction?
'COLPROT'                 Shall we discard for the prediction?
'ORIGPROT'                Shall we discard for the prediction?
'EXAMDATE'                Shall we discard for the prediction?
'DX_bl'                   Hotone, categorical
'AGE',                    OK, int
'PTGENDER'                Hotone, categorical
'PTEDUCAT'                OK, int
'PTETHCAT',               Hotone, categorical
'PTRACCAT',               Hotone, categorical 
'PTMARRY'                 Hotone, categorical
'APOE4'                   Hotone, categorical   Class of protein
'CDRSB'                   OK,float              Clinical Dementia Rating Scale–Sum of Boxes
'ADAS11'                  OK,float              Alzheimer's Disease Assessment Scale-Cognitive Subscale
'ADAS13'                  OK,float              Alzheimer's Disease Assessment Scale-Cognitive Subscale
'MMSE',                   OK,float              Mini Mental State Examination
'RAVLT_immediate'         OK,float              Rey Auditory Verbal Learning Test
'RAVLT_learning'          OK,float              Rey Auditory Verbal Learning Test
'RAVLT_forgetting',       OK,float              Rey Auditory Verbal Learning Test
'RAVLT_perc_forgetting'   OK,float              Rey Auditory Verbal Learning Test
'FAQ'                     OK,float              Functional Assessment Questionnaire
'MOCA'                    OK,float              Montreal Cognitive Assessment (MoCA)
'EcogPtMem'               OK,float              Cognitive
'EcogPtLang',             OK,float              Cognitive
'EcogPtVisspat'           OK,float              Cognitive
'EcogPtPlan'              OK,float              Cognitive
'EcogPtOrgan'             OK,float              Cognitive
'EcogPtDivatt'            OK,float              Cognitive
'EcogPtTotal'             OK,float              Cognitive
'EcogSPMem'               OK,float              Cognitive
'EcogSPLang'              OK,float              Cognitive
'EcogSPVisspat'           OK,float              Cognitive
'EcogSPPlan',             OK,float              Cognitive
'EcogSPOrgan'             OK,float              Cognitive
'EcogSPDivatt'            OK,float              Cognitive
'EcogSPTotal'             OK,float              Cognitive
'FSVERSION',              Just one value, discard
'Hippocampus'             OK,float              Hippocampus volume
'WholeBrain'              OK,float              Brain volume
'ICV'                     OK,float              Intracranial volume
'DX'                      OUR Y, ENCODING
'EXAMDATE_bl'             Shall we discard for the prediction?
'CDRSB_bl'                       ***FROM HERE BASELINE***
'ADAS11_bl'
'ADAS13_bl'
'MMSE_bl'
'RAVLT_immediate_bl'
'RAVLT_learning_bl'
'RAVLT_forgetting_bl'
'RAVLT_perc_forgetting_bl',
'FAQ_bl', 'FLDSTRENG_bl'
'FSVERSION_bl'
'Ventricles_bl',
'Hippocampus_bl'
'WholeBrain_bl'
'Entorhinal_bl'
'Fusiform_bl',
'MidTemp_bl'
'ICV_bl'
'MOCA_bl'
'EcogPtMem_bl'
'EcogPtLang_bl',
'EcogPtVisspat_bl'
'EcogPtPlan_bl'
'EcogPtOrgan_bl',
'EcogPtDivatt_bl'
'EcogPtTotal_bl'
'EcogSPMem_bl'
'EcogSPLang_bl',
'EcogSPVisspat_bl'
'EcogSPPlan_bl'
'EcogSPOrgan_bl',
'EcogSPDivatt_bl'
'EcogSPTotal_bl'
'FDG_bl'                   OK,float               fluorescence connect to PET
'AV45_bl'                  OK,float               average survival
'Years_bl',                OK,float               years form baseline
'Month_bl'                 Shall we discard for the prediction? We already have 'Years_bl'
'Month'                    Shall we discard for the prediction? We already have 'Years_bl'
'M'                        Shall we discard for the prediction? We already have 'Years_bl'
'update_stamp              Shall we discard for the prediction? 